<h1><center><font color='blue'>TAREA FINAL</font></center></h1>

## Introduction


Construiremos un clasificador de imágenes usando el modelo pre-entrenado VGG16 y compararemos su performance con el ResNet50.

## Tabla de contenido

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Descargar los datos
2. <a href="#item42">Parte 1</a>
3. <a href="#item43">Parte 2</a>  
4. <a href="#item44">Parte 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Descargar los datos

In [1]:
## get the data
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

import urllib.request
url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip'
filename = 'concrete_data_week4.zip'
urllib.request.urlretrieve(url, filename) 

('concrete_data_week4.zip', <http.client.HTTPMessage at 0x1f26e331d30>)

In [2]:
#!unzip concrete_data_week3.zip

import zipfile
with zipfile.ZipFile('concrete_data_week4.zip', 'r') as zip_ref:
    zip_ref.extractall()

Luego de descomprimir los datos verá que los mismos ya han sido divididos en conjuntos de entrenamiento, validación y tests.


<a id="item42"></a>

## Parte 1

Diseñaremos un clasificador usando el modelo pre-entrenado VGG16.

Construiremos el clasificador como sigue:

1. Importaremos las librerías, módulos y paquetes necesarios. En particular importaremos *preprocess_input* desde <code>keras.applications.vgg16</code>.
2. Usaremos un tamaño de lote de 100 imágenes tanto para entrenamiento como para validación.
3. Construiremos un ImageDataGenerator para los conjuntos de entrenamiento y validación. VGG16 fue entrenado originalmente con imágenes 224x224, recordaremos esto al crear las instancias ImageDataGenerator.
4. Crearemos un modelo secuencial. Le agregaremos VGG16 al modelo y un una capa densa.
5. Compilaremos el modelo usando el optimizador adam y la categorical_crossentropy como función de pérdida.
6. Ajustaremos el modelo.


In [11]:
# Imoport libraries
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50,vgg16,VGG16
from keras.applications.vgg16 import preprocess_input

In [4]:
# variables
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [5]:
# image data generator train and validation
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)
#
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')
#
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [12]:
# build model
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(num_classes, activation='softmax'))
model.layers
model.layers[0].trainable = False
model.summary()

58892288/58889256 [==============================] - 8s 0us/step
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [13]:
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# fit the model
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2
#
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
301/301 [==============================] - 5666s 19s/step - loss: 0.2339 - accuracy: 0.9227 - val_loss: 0.0357 - val_accuracy: 0.9921
Epoch 2/2
301/301 [==============================] - 6140s 20s/step - loss: 0.0270 - accuracy: 0.9938 - val_loss: 0.0198 - val_accuracy: 0.9956


In [16]:
model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Parte 2

Evaluaremos los modelos sobre los datos de test. Haremos lo siguiente:

1. Cargaremos el modelo ResNet50 que construimos en el lab anterior.
2. Construiremos un ImageDataGenerator para el conjunto de test. Para esta instancia, solamente debe pasársele el directorio de las imágenes de test, el target_size y el parámetro **shuffle** debe establecerse en False.
3. Usaremos el método **evaluate_generator** para evaluar los modelos sobre los datos de test, pasando el ImageDataGenerator anterior como argumento.
4. Imprimiremos la performance del clasificador usando el modelo pre-entrenado VGG16.
5. Imprimiremos la performance del clasificador usando el modelo pre-entrenado ResNet50.



In [17]:
# Load the model
from keras.models import load_model
saved_model = load_model("classifier_resnet_model.h5")

In [18]:
# Image data generator for the test set
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 500 images belonging to 2 classes.


## VGG16

In [19]:
# VGG 16
aux = model.evaluate_generator(test_generator)


Instructions for updating:
Please use Model.evaluate, which supports generators.


In [65]:
print("loss: %.3f - acc: %.3f" % (aux[0], aux[1]))

loss: 0.017 - acc: 0.996


## RESNET50

In [21]:
aux2 = saved_model.evaluate_generator(test_generator)

In [69]:
print("loss: %.3f - acc: %.3f" % (aux2[0], aux2[1]))

loss: 0.002 - acc: 1.000


<a id="item44"></a>

## Parte 3

Predeciremos si las imágenes de test tienen grietas o no. Haremos lo siguiente:

1. Usaremos el método **predict_generator** para predecir la clase de las imágenes en el conjunto de test, pasando la instancia ImageDataGenerator de los datos de test definida previamente.

2. Reportaremos las predicciones de las primeras 5 imágenes. Imprimiremos algo como esto:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

## 1. VGG16

In [27]:
predict = model.predict_generator(test_generator)

Instructions for updating:
Please use Model.predict, which supports generators.


In [72]:
import numpy as np\

predicted_class=np.argmax(predict,axis=1)
for i in range(0,5):
    if predicted_class[i] == 0:
        print('Negative')
    else:
        print('Positive')

Negative
Negative
Negative
Negative
Negative


## 2. RESNET50


In [58]:
predictResnet = saved_model.predict_generator(test_generator)

In [74]:
predicted_classResnet=np.argmax(predictResnet,axis=1)
for i in range(0,5):
    if predicted_classResnet[i] == 0:
        print('Negative')
    else:
        print('Positive')

Negative
Negative
Negative
Negative
Negative
